# De Bruijn graphs

You will be implementing one of the primary assembly algorithms from short-read data that is used today. We will implement a simple form of the algorithm where we _assume perfect sequencing_. That is, everything is sequenced exactly once and there are no errors or variants in the sequencing. 

A graph is composed of **nodes** and **edges** and we will need to develop a data strcture to track edges between nodes in our graph. We have provided the basic class structure as well as descriptions of functions to `add_edge` and `remove_edge` from the graph. You will need to implement these functions in order to then build the de Bruijn graph. 

In our implementation below, we use a `defaultdict` data structure to hold a list of all edges in the graph where all "right" nodes connected to a "left" node are stored in a list for that node.

```
build_debruijn_graph:
define substring length k and input string
For each k-length substring of input:
  split k mer into left and right k-1 mer
  add k-1 mers as nodes with a directed edge from left k-1 mer to right k-1 mer
```

---
## Eulerian walk

To continue our implementation from last class, we will use our De Bruijn graph to output a valid sequence from the assembly. This is implemented as a recursive algorithm by considering all valid edges. You will notice that as you change $k$, we are able to better recapitulate our sequence depending on how repetitive it is. In a more complex implementation of a Eulerian walk there are heuristics and defined rules for determining the validity of traversing a specific edge in the graph to result in a full graph-traversal. One of these methods is to traverse the graph in a depth first manner to avoid sectioning off any part of the graph in the traversal. In our implementation we will ignore these for simplicity.

```
eulerian_walk:
Beginning at first_node as node

For node:
    follow a random valid edge from node
    remove edge
    recurse
```


---
# Toy Example

In [6]:
import os
print(os.getcwd())
from utils import read_fastq, assemble_mouse_genome


/Users/chanteralazard/Projects/Northeastern_Masters/BINF6250/EmpireOfDirt/project04


In [1]:
print("="*60)
print("EXAMPLE 1: Assembling 9 overlapping reads into one contig")
print("="*60)

toy_reads_1 = [
    "ATGGCGTACG",  # Read 1
    "GGCGTACGTT",  # Read 2: overlaps with Read 1
    "CGTACGTTAC",  # Read 3: overlaps with Read 2
    "TACGTTACCA",  # Read 4: overlaps with Read 3
    "CGTTACCATG",  # Read 5: overlaps with Read 4
    "TTACCATGGG",  # Read 6: overlaps with Read 5
    "ACCATGGGCC",  # Read 7: overlaps with Read 6
    "CATGGGCCTA",  # Read 8: overlaps with Read 7
    "TGGGCCTAAA"   # Read 9: overlaps with Read 8
]

print(f"\nInput: {len(toy_reads_1)} reads")
print("First read:  ", toy_reads_1[0])
print("Last read:   ", toy_reads_1[-1])
print(f"\nBuilding De Bruijn graph with k=6...")

from de_bruijn_module import DeBruijnGraph
toy_graph = DeBruijnGraph(toy_reads_1, 6, 43375)
toy_graph_node = toy_graph.get_starting_node()
#print(toy_graph_node)
euler_walk_toy = toy_graph.eulerian_walk(toy_graph_node, toy_graph.graph)
print(f"\n{euler_walk_toy}")

EXAMPLE 1: Assembling 9 overlapping reads into one contig

Input: 9 reads
First read:   ATGGCGTACG
Last read:    TGGGCCTAAA

Building De Bruijn graph with k=6...
defaultdict(<class 'list'>, {'TGGCG': ['ATGGC'], 'GGCGT': ['TGGCG'], 'GCGTA': ['GGCGT', 'GGCGT'], 'CGTAC': ['GCGTA', 'GCGTA'], 'GTACG': ['CGTAC', 'CGTAC', 'CGTAC'], 'TACGT': ['GTACG', 'GTACG'], 'ACGTT': ['TACGT', 'TACGT', 'TACGT'], 'CGTTA': ['ACGTT', 'ACGTT'], 'GTTAC': ['CGTTA', 'CGTTA', 'CGTTA'], 'TTACC': ['GTTAC', 'GTTAC'], 'TACCA': ['TTACC', 'TTACC', 'TTACC'], 'ACCAT': ['TACCA', 'TACCA'], 'CCATG': ['ACCAT', 'ACCAT', 'ACCAT'], 'CATGG': ['CCATG', 'CCATG'], 'ATGGG': ['CATGG', 'CATGG', 'CATGG'], 'TGGGC': ['ATGGG', 'ATGGG'], 'GGGCC': ['TGGGC', 'TGGGC', 'TGGGC'], 'GGCCT': ['GGGCC', 'GGGCC'], 'GCCTA': ['GGCCT', 'GGCCT'], 'CCTAA': ['GCCTA'], 'CTAAA': ['CCTAA']})

[np.str_('ATGGC'), np.str_('TGGCG'), np.str_('GGCGT'), np.str_('GCGTA'), np.str_('CGTAC'), np.str_('GTACG'), np.str_('TACGT'), np.str_('ACGTT'), np.str_('CGTTA'), np.str_(

In [6]:
print(toy_graph_node)

GTACG


In [7]:
print(toy_graph.tour_to_sequence(euler_walk_toy))
#ATGGCGTACGTTACCATG
print(f"\nLength of tour: {len(toy_graph.tour_to_sequence(euler_walk_toy))}")

ATGGCGTACG

Length of tour: 10


In [2]:
contigs = toy_graph.assemble_contigs(43375)
print(contigs)


['GGCGTACGTTACCA', 'TTACCATGGGCCTA', 'TACGTT', 'CATGGGCC', 'ACCATG', 'CGTTAC', 'TGGGCCTAAA', 'CGTACG']


In [3]:
toy_graph.get_assembly_stats(contigs)
toy_graph.write_fasta(contigs, "toy_assembly.fasta", "assembled_contigs")

---
# Real data
This section utilizes a FASTQ file with 10 million "perfect" 150 bp reads simulated
from the mouse genome (`GRCm39`) and tasks your program to ingest, assemble, and traverse
the genome with statistical output report.

In [ ]:
"""Driver program for mouse genome assembly using De Bruijn graphs.

This script demonstrates how to use the completed DeBruijnGraph class to
assemble 10 million perfect 150bp single-end reads from the mouse genome.

Usage:
    Run all cells in order in a Jupyter notebook.

Expected input:
    - File: mouse_SE_150bp.fq
    - Format: FASTQ
    - Reads: 10 million perfect 150bp single-end reads
    - Source: Simulated from mouse genome using wgsim

Output:
    - mouse_assembly.fasta: Assembled contigs
    - mouse_assembly_stats.txt: Detailed statistics
"""


print("\n" + "="*80)
print("MOUSE GENOME DE BRUIJN GRAPH ASSEMBLY")
print("Student Assignment Driver Program")
print("="*80 + "\n")

# Run the assembly pipeline
result = assemble_mouse_genome()

# Display sample contigs
print("="*80)
print("SAMPLE ASSEMBLED CONTIGS")
print("="*80 + "\n")

contigs = result['contigs']
for i in range(min(5, len(contigs))):
    contig = contigs[i]
    preview = contig[:100] + "..." if len(contig) > 100 else contig
    print(f">contig_{i+1} length={len(contig)}")
    print(preview)
    print()

print("="*80)
print("✓ ASSEMBLY COMPLETE")
print("="*80)
print(f"\nResults saved to:")
print(f"  • mouse_assembly.fasta - {result['stats']['num_contigs']:,} assembled contigs")
print(f"  • mouse_assembly_stats.txt - Detailed assembly statistics")
print(f"\nKey metrics:")
print(f"  • Total assembly: {result['stats']['total_length']:,} bp")
print(f"  • N50: {result['stats']['n50']:,} bp")
print(f"  • Longest contig: {result['stats']['longest_contig']:,} bp")
print(f"  • Coverage: {result['coverage']:.1f}x")
print()


MOUSE GENOME DE BRUIJN GRAPH ASSEMBLY
Student Assignment Driver Program

MOUSE GENOME ASSEMBLY PIPELINE
Start time: 2026-02-18 12:34:18

STEP 1: Loading sequencing reads
--------------------------------------------------------------------------------
Input file: data/mouse_SE_150bp.fq
Expected: 10 million reads, 150bp each



NameError: name 'read_fastq' is not defined